In [1]:
%%bigquery
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.38rows/s]


,ontime,dep_delay,taxi_out,distance,is_eval_day
0,ontime,-5.0,19.0,287.0,True
1,ontime,-9.0,10.0,1133.0,False
2,ontime,-2.0,9.0,889.0,False
3,ontime,-8.0,24.0,123.0,False
4,late,78.0,9.0,204.0,False


In [2]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_lm
OPTIONS(input_label_cols=['ontime'],
        model_type='logistic_reg',
        data_split_method='custom',
        data_split_col='is_eval_day')
AS
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1279.92query/s]                        


""


In [3]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL dsongcp.arr_delay_lm)

Downloading: 100%|██████████| 11/11 [00:01<00:00,  8.09rows/s]


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,10,0.169790,0.167233,25.6,22397
1,0,9,0.171054,0.168219,25.6,22573
2,0,8,0.173285,0.170453,51.2,21947
3,0,7,0.190027,0.186490,25.6,21234
4,0,6,0.219350,0.214753,12.8,21826
5,0,5,0.261108,0.255400,6.4,23362
6,0,4,0.321923,0.316414,3.2,22478
7,0,3,0.415231,0.411193,1.6,22508
8,0,2,0.521449,0.519102,0.8,23450
9,0,1,0.606936,0.605817,0.4,23181


In [4]:
%%bigquery
SELECT *
FROM ML.EVALUATE(MODEL dsongcp.arr_delay_lm,
                 (
                     
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
    is_train_day = 'False'
                     
                 ),
                 STRUCT(0.7 AS threshold))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.36s/rows]


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.964337,0.956535,0.935174,0.96042,0.167233,0.956292


In [5]:
%%bigquery
SELECT * FROM ML.WEIGHTS(MODEL dsongcp.arr_delay_lm)

Downloading: 100%|██████████| 4/4 [00:01<00:00,  3.18rows/s]


,processed_input,weight,category_weights
0,dep_delay,-0.132984,[]
1,taxi_out,-0.121715,[]
2,distance,0.000223,[]
3,__INTERCEPT__,4.762572,[]


In [6]:
%%bigquery
SELECT * FROM ML.PREDICT(MODEL dsongcp.arr_delay_lm,
                        (
SELECT 12.0 AS dep_delay, 14.0 AS taxi_out, 1231 AS distance
                        ))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.42s/rows]


,predicted_ontime,predicted_ontime_probs,dep_delay,taxi_out,distance
0,ontime,"[{'label': 'ontime', 'prob': 0.850350772376715...",12.0,14.0,1231


In [7]:
%%bigquery
WITH predictions AS (
SELECT
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),
stats AS (
SELECT
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)
SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.26s/rows]


,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.836363,1301948,0.964337,283750,0.213091


In [ ]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_airports_lm
OPTIONS(input_label_cols=['ontime'],
        model_type='logistic_reg',
        data_split_method='custom',
        data_split_col='is_eval_day')
AS
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False

Executing query with job ID: 3a946bfa-3ecb-4a8b-97fb-3088fea4ccac
Query executing: 21.71s

In [ ]:
%%bigquery
SELECT *
FROM ML.EVALUATE(MODEL dsongcp.arr_delay_airports_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))

In [ ]:
%%bigquery
SELECT *
FROM ML.EVALUATE(MODEL dsongcp.arr_delay_airports_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))